In [ ]:
!pip install --upgrade -r requirements.txt

In [1]:
!TOKENIZERS_PARALLELISM=true wget -O biblioteca.pdf "https://anexosportal.datalegis.net/arquivos/1878261.pdf"

'TOKENIZERS_PARALLELISM' nÆo ‚ reconhecido como um comando interno
ou externo, um programa oper vel ou um arquivo em lotes.


## Código para popular um Banco de Dados Vetorial

In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

dados = PyPDFLoader("Dados/D11246.pdf").load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=100)
textos = text_splitter.split_documents(dados)
embedding_engine = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
vector_db = Chroma.from_documents(textos, embedding_engine)

## Código para montar um RAG

In [7]:
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
import os
load_dotenv()

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
n_documentos = 1

def format_docs(documentos):
    return "\n\n".join(documento.page_content for documento in documentos)

llm = ChatOpenAI(openai_api_key = OPENAI_API_KEY, model = "gpt-4o-mini")
prompt = hub.pull("rlm/rag-prompt")

rag = (
    {
        "question": RunnablePassthrough(),
        "context": vector_db.as_retriever(k = n_documentos) 
                    | format_docs
    }
    | prompt
    | llm
    | StrOutputParser()
)

C:\Users\jeferson.oliveira\AppData\Local\Programs\Python\Python313\Lib\site-packages\langsmith\client.py:280: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


## Código para perguntar ao RAG

In [1]:
prompt = "Você conhece o decreto nº 11246?"
rag.invoke(prompt)

NameError: name 'rag' is not defined